# Select the LCLS-II py3 kernel in the top right
# Import libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import psana as ps

# Specifiy run

In [ ]:
exp = 'tmolv2918'#the experiment
run_number = 122 # the run within an experiment
ds = ps.DataSource(exp=exp, run=run_number)
run = next(ds.runs())

In [ ]:
print(run.detnames)

# specify detector objects

In [ ]:
andor = run.Detector('andor')
gmd = run.Detector('gmd') # gmd before the attenuator
xgmd = run.Detector('xgmd') # gmd after the attenuator
timing = run.Detector('timing') 
hsd = run.Detector('hsd') # ion tof, Ktof and so on
ebeam = run.Detector('ebeam')# this gives the mean photon energy, is a data extracted from the electron beam 
und42K = run.Detector('UND_39_k')
epics_var = run.Detector('epicsinfo')

# load some data

In [ ]:
# User specified inputs

loadImages='false' #if this is false then no images will be read (to save memory)

Nfind = 2000
nskip = 1
nupdate = 100
channelItof = 0 # ion ToF
channelKtof = 6 # channel for the Ktof
channelLaser = 9# according to Matt that is the ADC channel for the laser diode at the outcoupling port

# Initialize arrays
energies = np.empty(Nfind) #gmd
xenergies = np.empty(Nfind) #xgmd

photonEs = np.empty(Nfind)
undKs = np.empty(Nfind)
delays = np.empty(Nfind)
evrs = None
tItof = None #time axis
tKtof = None#time axis
laserDiode = None#time axis

wfs = None
Ktof_spec = None
laserDiodeArray=None
andorArray = np.empty((Nfind, 2048))

imsVMI = None
imsTimeTool = None

# Event iteration
Nfound = 0
for nevent, event in enumerate(run.events(),start=0):
    
    if nevent%nupdate==0: print(nevent, Nfound)
    if nevent%nskip!=0: continue
  
    #undK = und42K(event)
    #if undK is None: continue
    energy = gmd.raw.energy(event)
    if energy is None:
        print("Didn't find GMD")
    #    continue
    xenergy = xgmd.raw.energy(event)
    if energy is None:
        print("Didn't find GMD")
    #    continue
    evr = timing.raw.eventcodes(event)
    if evr is None:
        print("Didn't find EVR")
        continue
        
    if loadImages=='true':
        imVMI = opalVMI.raw.image(event)
        if imVMI is None:
            print("Didn't find Opal")
            continue
        imTimeTool = opalTimeTool.raw.image(event)
        if imTimeTool is None:
            print("Didn't find Opal")
            continue
            
    andor_data=andor.raw.value(event)        
    if andor_data is None:
        print("Didn't find Andor")
        continue
        
    hsd_data = hsd.raw.waveforms(event)
    if hsd_data is None:
        print("Didn't find HSD")
    #    continue
    photonE = ebeam.raw.ebeamPhotonEnergy(event)
    if photonE is None:
        print("Didn't find photon energy")
    #    continue
    
    energies[Nfound] = energy
    xenergies[Nfound] = xenergy
    photonEs[Nfound] = photonE
    andorArray[Nfound]=andor_data.reshape((2048,))

    
    #undKs[Nfound] = undK
    
    if evrs is None:
        evrs = np.empty((Nfind, len(evr)))
    evrs[Nfound] = evr
    
    if tKtof is None:
        tKtof = hsd_data[channelKtof]['times'][::4] * 1e6
        Ktof_spec = np.empty((Nfind, len(tKtof)))
    Ktof_spec[Nfound] = hsd_data[channelKtof][0][::4]
    
    
    if laserDiode is None:
        laserDiode = hsd_data[channelLaser]['times'][::4] * 1e6
        laserDiodeArray = np.empty((Nfind, len(laserDiode)))
    laserDiodeArray[Nfound] = hsd_data[channelLaser][0][::4]
    
    if tItof is None:
        tItof = hsd_data[channelItof]['times'][::4] * 1e6
        wfs = np.empty((Nfind, len(tItof)))
    wfs[Nfound] = hsd_data[channelItof][0][::4]
    
    
    if loadImages=='true':
        if imsVMI is None:
            imsVMI = np.empty((Nfind,)+imVMI.shape)
        imsVMI[Nfound] = imVMI

        if imsTimeTool is None:
            imsTimeTool = np.empty((Nfind,)+imTimeTool.shape)
        imsTimeTool[Nfound] = imTimeTool
        
        
   # delays[Nfound]=delay(event) # reading the relative delay of UV and x-rays
    
    

    
    Nfound += 1
  
    if Nfound==Nfind: break

# get some evrs

In [ ]:
gas_on = evrs[:,70]==1
gas_off = evrs[:,71]==1
UV_on = evrs[:,67]==1
UV_off = evrs[:,68]==1 #goose?
Xray_off = evrs[:,161]==1 #BYKIK, makes the x-rays go off
Xray_on = evrs[:,161]==0

In [ ]:
for counter in range(500):
    plt.plot(laserDiodeArray[counter][0:2000])

determine background

In [ ]:
backgroundArray= laserDiodeArray.mean(0)[2000:8000]
background=backgroundArray.mean(0)
laserDiodeArray=laserDiodeArray-background

In [ ]:

plt.plot(laserDiode[0:1500],laserDiodeArray[UV_on].mean(0)[0:1500])
plt.plot(laserDiode[0:1500],laserDiodeArray[UV_off].mean(0)[0:1500])
plt.xlabel('time [ $\mu$s]')
plt.ylabel('Diode signal [arb.]')
plt.title('means of diodes first 500 shots in run 160')
plt.legend(['UV_on','UV_off - goose'])

In [ ]:

plt.plot(laserDiode[0:1500],laserDiodeArray[UV_on].mean(0)[0:1500])
plt.plot(laserDiode[0:1500],laserDiodeArray[UV_off].mean(0)[0:1500])
plt.xlabel('time [ $\mu$s]')
plt.ylabel('Diode signal [arb.]')
plt.title('means of diodes first 20000 shots in run'+str(run_number))
plt.legend(['UV_on','UV_off - goose'])
plt.xlim((0.2,0.3))

# integration of diode signal

In [ ]:
plt.plot(abs(laserDiodeArray[UV_on]).mean(0)[0:1500])
plt.plot(abs(laserDiodeArray[UV_off]).mean(0)[0:1500])

In [ ]:
lowerBound=200
upperBound=1200

UVint=np.zeros(Nfind)
for counter in range(Nfind):
    
    UVint[counter]=np.sum(abs(laserDiodeArray[counter])[lowerBound:upperBound])

In [ ]:
r=np.linspace(0,Nfind-1,Nfind)
print(r[UV_on])

In [ ]:
r=np.linspace(0,Nfind-1,Nfind)#simply an integer array
plt.plot(r[UV_on],UVint[UV_on])
plt.plot(r[UV_off],UVint[UV_off])
plt.xlabel('shot')
plt.ylabel('UV pulse energy [arb.]')
plt.title('run '+str(run_number))
plt.legend(['UV_on','UV_off - goose'])

In [ ]:
plt.hist(UVint, bins=50)
plt.xlabel("UV pulse energy [arb.]")
plt.ylabel('# shots in bin')
plt.title('run'+str(run_number))

In [ ]:
good=Xray_on & ~np.isnan(xenergies) & ~np.isnan(UVint)
plt.hist2d(xenergies[good]*1000, UVint[good],bins=20);
plt.title('run '+str(run_number))
plt.xlabel('x-ray pulse energy [$\mu$J]')
plt.ylabel('UV pulse energy [arb.]')

In [ ]:
xenergies[Xray_on]